## Part 3: Data exchange between Julia and Python

[AwkwardArray.jl](https://github.com/JuliaHEP/AwkwardArray.jl) package - Awkward Array in Julia mirrors the Python library, enabling effortless zero-copy data exchange between Julia and Python.

In [1]:
from juliacall import Main as jl

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


#### 1. Write Julia code in a sepatate file:

```julia
using AwkwardArray

function f1(x)
  print(typeof(x))
  return AwkwardArray.convert(x)
end;
```

In [2]:
%cat test_funcs.jl

using AwkwardArray

function f1(x)
  print(typeof(x))
  return AwkwardArray.convert(x)
end;

#### 2. Include Julia code from a sepatate Julia file:

In [3]:
jl.include('test_funcs.jl')

f1 (generic function with 1 method)

#### Note: this is equivalent to executing the following two cells

Julia code is written as Python strings.

#### 3. Pass Python Awkward Array to Julia function:

In [4]:
import awkward as ak
events = ak.from_parquet("./data/SMHiggsToZZTo4L.parquet")

Let's check the data type:

In [5]:
events.show(type=True)

type: 299973 * {
    run: int32,
    luminosityBlock: int64,
    event: uint64,
    PV: Vector3D[
        x: float32,
        y: float32,
        z: float32
    ],
    electron: var * Momentum4D[
        pt: float32,
        eta: float32,
        phi: float32,
        mass: float32,
        charge: int32,
        pfRelIso03_all: float32,
        dxy: float32,
        dxyErr: float32,
        dz: float32,
        dzErr: float32
    ],
    muon: var * Momentum4D[
        pt: float32,
        eta: float32,
        phi: float32,
        mass: float32,
        charge: int32,
        pfRelIso03_all: float32,
        pfRelIso04_all: float32,
        dxy: float32,
        dxyErr: float32,
        dz: float32,
        dzErr: float32
    ],
    MET: Momentum2D[
        pt: float32,
        phi: float32
    ]
}
[{run: 1, luminosityBlock: 156, event: 46501, PV: {...}, electron: [], ...},
 {run: 1, luminosityBlock: 156, event: 46502, PV: {...}, electron: [...], ...},
 {run: 1, luminosityBlock: 156,

In [6]:
arr = jl.f1(events)

AwkwardArray.RecordArray{(:run, :luminosityBlock, :event, :PV, :electron, :muon, :MET), Tuple{AwkwardArray.PrimitiveArray{Int32, SubArray{Int32, 1, Base.ReinterpretArray{Int32, 1, UInt8, Vector{UInt8}, false}, Tuple{UnitRange{Int64}}, true}, :default}, AwkwardArray.PrimitiveArray{Int64, SubArray{Int64, 1, Base.ReinterpretArray{Int64, 1, UInt8, Vector{UInt8}, false}, Tuple{UnitRange{Int64}}, true}, :default}, AwkwardArray.PrimitiveArray{Int64, SubArray{Int64, 1, Base.ReinterpretArray{Int64, 1, UInt8, Vector{UInt8}, false}, Tuple{UnitRange{Int64}}, true}, :default}, AwkwardArray.RecordArray{(:x, :y, :z), Tuple{AwkwardArray.PrimitiveArray{Float32, SubArray{Float32, 1, Base.ReinterpretArray{Float32, 1, UInt8, Vector{UInt8}, false}, Tuple{UnitRange{Int64}}, true}, :default}, AwkwardArray.PrimitiveArray{Float32, SubArray{Float32, 1, Base.ReinterpretArray{Float32, 1, UInt8, Vector{UInt8}, false}, Tuple{UnitRange{Int64}}, true}, :default}, AwkwardArray.PrimitiveArray{Float32, SubArray{Float32,

In [7]:
arr

<Array [{run: 1, ...}, ..., {run: 1, ...}] type='299973 * {run: int32, lumi...'>

In [8]:
type(arr)

awkward.highlevel.Array

In [9]:
arr.show(type=True)

type: 299973 * {
    run: int32,
    luminosityBlock: int64,
    event: int64,
    PV: Vector3D[
        x: float32,
        y: float32,
        z: float32
    ],
    electron: var * Momentum4D[
        pt: float32,
        eta: float32,
        phi: float32,
        mass: float32,
        charge: int32,
        pfRelIso03_all: float32,
        dxy: float32,
        dxyErr: float32,
        dz: float32,
        dzErr: float32
    ],
    muon: var * Momentum4D[
        pt: float32,
        eta: float32,
        phi: float32,
        mass: float32,
        charge: int32,
        pfRelIso03_all: float32,
        pfRelIso04_all: float32,
        dxy: float32,
        dxyErr: float32,
        dz: float32,
        dzErr: float32
    ],
    MET: Momentum2D[
        pt: float32,
        phi: float32
    ]
}
[{run: 1, luminosityBlock: 156, event: 46501, PV: {...}, electron: [], ...},
 {run: 1, luminosityBlock: 156, event: 46502, PV: {...}, electron: [...], ...},
 {run: 1, luminosityBlock: 156, 

## Faster, more efficient data processing

Let's combine Python and Julia:

In [10]:
one_event = jl.first(events)

In [11]:
one_event

{run: 1, luminosityBlock: 156, event: 46501, PV: {x: ..., ...}, ...}

In [12]:
type(one_event)

juliacall.AnyValue

In [13]:
jl.one_event = one_event

In [14]:
events.muon

<Array [[{pt: 63, eta: -0.719, ...}, ...], ...] type='299973 * var * Moment...'>

In [15]:
jl.Muon_pt = events.muon.pt

In [16]:
jl.Muon_eta = events.muon.eta

In [17]:
jl.Muon_phi = events.muon.phi

In [18]:
jl.Muon_mass = events.muon.mass

In [19]:
jl.Muon_charge = events.muon.charge

In [20]:
jl.Muon_pt

299973-element AwkwardArray.ListOffsetArray{SubArray{Int64, 1, Base.ReinterpretArray{Int64, 1, UInt8, Vector{UInt8}, false}, Tuple{UnitRange{Int64}}, true}, AwkwardArray.PrimitiveArray{Float32, SubArray{Float32, 1, Base.ReinterpretArray{Float32, 1, UInt8, Vector{UInt8}, false}, Tuple{UnitRange{Int64}}, true}, :default}, :default}:
 [63.04387f0, 38.120346f0, 4.0486875f0]
 0-element AwkwardArray.PrimitiveArray{Float32, Vector{Float32}, :default}
 0-element AwkwardArray.PrimitiveArray{Float32, Vector{Float32}, :default}
 [54.33275f0, 23.515282f0, 52.871075f0, ..., 8.393386f0, 3.4901235f0]
 0-element AwkwardArray.PrimitiveArray{Float32, Vector{Float32}, :default}
 [38.503757f0, 47.00221f0]
 [4.453538f0]
 0-element AwkwardArray.PrimitiveArray{Float32, Vector{Float32}, :default}
 0-element AwkwardArray.PrimitiveArray{Float32, Vector{Float32}, :default}
 0-element AwkwardArray.PrimitiveArray{Float32, Vector{Float32}, :default}
 ⋮
 [37.18836f0, 50.064648f0]
 [43.15527f0, 23.953928f0]
 [24.2498

In [21]:
jl.first(events.muon.charge, 4)

4-element AwkwardArray.ListOffsetArray{Vector{Int64}, AwkwardArray.PrimitiveArray{Int32, SubArray{Int32, 1, Base.ReinterpretArray{Int32, 1, UInt8, Vector{UInt8}, false}, Tuple{UnitRange{Int64}}, true}, :default}, :default}:
 [1, -1, 1]
 0-element AwkwardArray.PrimitiveArray{Int32, Vector{Int32}, :default}
 0-element AwkwardArray.PrimitiveArray{Int32, Vector{Int32}, :default}
 [1, -1, -1, 1, -1, -1, 1]

Awkward Array implements the convertion rules between Python and Julia Awkward types:

In [22]:
events_from_julia = jl.AwkwardArray.convert(events)

In [23]:
type(events_from_julia)

awkward.highlevel.Array

In [24]:
events_from_julia.show(type=True)

type: 299973 * {
    run: int32,
    luminosityBlock: int64,
    event: int64,
    PV: Vector3D[
        x: float32,
        y: float32,
        z: float32
    ],
    electron: var * Momentum4D[
        pt: float32,
        eta: float32,
        phi: float32,
        mass: float32,
        charge: int32,
        pfRelIso03_all: float32,
        dxy: float32,
        dxyErr: float32,
        dz: float32,
        dzErr: float32
    ],
    muon: var * Momentum4D[
        pt: float32,
        eta: float32,
        phi: float32,
        mass: float32,
        charge: int32,
        pfRelIso03_all: float32,
        pfRelIso04_all: float32,
        dxy: float32,
        dxyErr: float32,
        dz: float32,
        dzErr: float32
    ],
    MET: Momentum2D[
        pt: float32,
        phi: float32
    ]
}
[{run: 1, luminosityBlock: 156, event: 46501, PV: {...}, electron: [], ...},
 {run: 1, luminosityBlock: 156, event: 46502, PV: {...}, electron: [...], ...},
 {run: 1, luminosityBlock: 156, 

In [25]:
events.muon.show(type=True)

type: 299973 * var * Momentum4D[
    pt: float32,
    eta: float32,
    phi: float32,
    mass: float32,
    charge: int32,
    pfRelIso03_all: float32,
    pfRelIso04_all: float32,
    dxy: float32,
    dxyErr: float32,
    dz: float32,
    dzErr: float32
]
[[{pt: 63, eta: -0.719, phi: 2.97, mass: 0.106, charge: 1, ...}, ..., {...}],
 [],
 [],
 [{pt: 54.3, eta: -1.06, phi: -0.362, mass: 0.106, charge: 1, ...}, ..., {...}],
 [],
 [{pt: 38.5, eta: 0.315, phi: 2.05, mass: 0.106, charge: -1, ...}, {...}],
 [{pt: 4.45, eta: -0.986, phi: 1.12, mass: 0.106, charge: 1, ...}],
 [],
 [],
 [],
 ...,
 [{pt: 37.2, eta: 1.1, phi: -0.875, mass: 0.106, charge: -1, ...}, {...}],
 [{pt: 43.2, eta: 2.15, phi: -1.3, mass: 0.106, charge: 1, ...}, {...}],
 [{pt: 24.2, eta: 0.327, phi: -0.997, mass: 0.106, charge: -1, ...}, ...],
 [],
 [{pt: 9.81, eta: 2.07, phi: 1.66, mass: 0.106, charge: 1, ...}, {...}],
 [{pt: 32.6, eta: 1.11, phi: -0.981, mass: 0.106, charge: 1, ...}, {...}],
 [{pt: 4.32, eta: -2.09, ph

Let's go to the next [notebook](AwkwardArray_Julia_Python-part4.ipynb).